In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
df1 = pd.read_csv("sales_2023-06-01_2023-11-30 (3).csv") # archivo ventas
inventario_original_1 = pd.read_excel("inventario0412.xlsx") # archivo inventario

In [3]:
inventario_original_1 = inventario_original_1.loc[:, ~inventario_original_1.columns.duplicated()]

In [4]:
inventario_original = inventario_original_1.loc[~inventario_original_1['Vendor'].isin(['Bukz USA', 'Bukz España'])].copy()
inventario_original = inventario_original.loc[inventario_original['Type'].isin(['Libro', 'Libros', 'Libros impresos'])]
inventario_original['ID'] = inventario_original['ID'].astype('Int64')

In [5]:
# Calcula el promedio de ventas basado en el número de meses de creación
def compute_sales_based_on_creation(row):
    total_sales = row[months].sum()
    return total_sales / row['meses de creación']

# Detecta si un producto ha tenido ventas de manera intermitente (i.e., ha tenido ventas, luego no ventas, y luego ventas nuevamente)
def is_intermittent(s):
    had_sales = False
    had_no_sales = False
    for sale in s:
        if sale > 0:
            if had_no_sales:
                return True
            had_sales = True
        else:
            if had_sales:
                had_no_sales = True
    return False

# Calcula el promedio de ventas para aquellos registros que tienen ventas intermitentes
def compute_average(s):
    months_with_sales = len(s[s > 0])
    if months_with_sales == 0:
        return 0
    return s.sum() / months_with_sales


# Calcula el promedio de ventas solo para los meses en los que hubo ventas
def compute_average_sales(row):
    months_with_sales = row[months].loc[row[months] > 0]
    return months_with_sales.mean()

# Calcula el promedio total de ventas considerando todos los meses
def compute_total_average(row, months):
    return row[months].sum() / len(months)

# Calcula el promedio de ventas de los últimos 6 meses
def compute_six_month_average(row):
    return row[months].mean()

# Asigna un valor sugerido y una descripción de caso a los registros del dataframe basado en una condición
def set_suggested_and_case(df, condition, suggested_value, case_description):
    df.loc[condition, 'sugerido'] = suggested_value
    df.loc[condition, 'tipo_caso'] = case_description
    return df

# Asigna un valor sugerido para libros antiguos con ventas bajas
def set_suggested_for_old_books(df):
    condition = (df['meses de creación'] >= 5) & (df['Total'].isin([1, 2]))
    return set_suggested_and_case(df, condition, 1, "Mas de 5 meses de creación y 2 o menos libros vendidos")

# Asigna un valor sugerido basado en el promedio de ventas para aquellos registros que han vendido durante 5 o 6 meses
def set_suggested_for_months_with_sales(df):
    condition = df['MonthsWithSales'].isin([5, 6])
    suggested_value = (df['Total'] / df['MonthsWithSales']).mul(1).round(0).astype(float)
    return set_suggested_and_case(df, condition, suggested_value, "5 o 6 meses de venta")

# Asigna un valor sugerido para productos creados recientemente
def set_suggested_for_recent_creation(df):
    condition = (df['meses de creación'] <= 4) & (df['sugerido'].isna())
    suggested_value = df.apply(compute_sales_based_on_creation, axis=1).round(0).fillna(0).astype(int)
    return set_suggested_and_case(df, condition, suggested_value, "Creado hace menos de 4 meses")

# Asigna un valor sugerido para productos con ventas intermitentes
def set_suggested_for_intermittent_sales(df, month_columns):
    df['intermittent'] = df[month_columns].apply(is_intermittent, axis=1)
    condition = df['intermittent'] & df['sugerido'].isna()
    suggested_value = df[month_columns].apply(compute_average, axis=1).mul(1).round(0)
    return set_suggested_and_case(df, condition, suggested_value, "Ventas intermitentes (posibles agotados)")

# Asigna un valor sugerido para productos que actualmente no tienen inventario
def set_suggested_for_no_inventory(df, inv_sede):
    condition = (df[inv_sede] == 0) & (df['August'] == 0) & df['sugerido'].isna()
    suggested_value = df.apply(compute_average_sales, axis=1).round(0)
    return set_suggested_and_case(df, condition, suggested_value, "Productos sin inventario en bodega actualmente")

# Asigna un valor sugerido para productos de reciente creación con ventas bajas
def set_suggested_for_recent_creation_with_low_sales(df):
    condition = (df['meses de creación'] < 5) & df['Total'].isin([1, 2, 3, 4]) & df['sugerido'].isna()
    return set_suggested_and_case(df, condition, 1, "Meses de creación menor a 5 y promedio de ventas 0,8 o menos, 1 predeterminado")

# Asigna un valor sugerido basado en el promedio de ventas de los últimos 6 meses
def set_suggested_for_average_sales(df, months):
    condition = (df['meses de creación'] > 4) & df['MonthsWithSales'].isin([3, 4]) & df['sugerido'].isna()
    suggested_value = df.apply(lambda row: compute_total_average(row, months), axis=1).round(0)
    return set_suggested_and_case(df, condition, suggested_value, "Promedio de ventas de 6 meses")


In [6]:
def compute_values(row):
    caso = row['tipo_caso']
    sugerido = row['sugerido']
    total = row['Total']

    # Por defecto, mantendremos sugerido_2 y meses de inventario como NaN
    sugerido_2 = np.nan
    meses_de_inventario = np.nan

    if caso == "5 o 6 meses de venta":
        sugerido_2 = sugerido * 4 if sugerido >= 5 else sugerido * 2
        meses_de_inventario = 4 if sugerido >= 5 else 2

    elif caso == "Creado hace menos de 4 meses":
        sugerido_2 = sugerido * 3 if sugerido >= 5 else sugerido * 2
        meses_de_inventario = 3 if sugerido >= 5 else 2

    elif caso == "Mas de 5 meses de creación y 2 o menos libros vendidos":
        sugerido_2 = sugerido
        meses_de_inventario = ""

    elif caso == "Meses de creación menor a 5 y promedio de ventas 0,8 o menos, 1 predeterminado":
        sugerido_2 = sugerido * 2
        meses_de_inventario = 2

    elif caso == "Productos sin inventario en bodega actualmente":
        sugerido_2 = sugerido * 2 if sugerido < 5 else sugerido * 3
        meses_de_inventario = 2 if sugerido < 5 else 3

    elif caso == "Promedio de ventas de 6 meses":
        sugerido_2 = sugerido * 2 if total >= 5 else sugerido
        meses_de_inventario = 2 if total >= 5 else ""

    elif caso == "Ventas intermitentes (posibles agotados)":
        sugerido_2 = sugerido * 2
        meses_de_inventario = 2

    return pd.Series([sugerido_2, meses_de_inventario], index=['sugerido_2', 'meses de inventario'])

In [7]:
sedes = ['Bukz Tesoro', 'Bukz Las Lomas', 'Bukz Mattelsa' ] 

resultados = {}  # Diccionario vacío para almacenar los DataFrames

for sede in sedes:
    inventario2 = inventario_original.copy()
    inv_sede = "Inventory Available: " + sede

    months = [ 'June', 'July', 'August', 'September','October', 'November']

    # Filtrar las filas que cumplan con el criterio
    df = df1.loc[~df1['product_vendor'].isin(['Bukz USA', 'Bukz España'])]
    df = df.loc[df['product_type'].isin(['Libro', 'Libros', 'Libros impresos'])]
    df["net_quantity"] = df["net_quantity"].apply(lambda x: abs(x))
    df.loc[df['net_quantity'] > 3, 'net_quantity'] = 1
    df['pos_location_name'] = df['pos_location_name'].replace('Libreria Provenza', 'Bukz Las Lomas')
    df = df.loc[df['api_client_title'].isin(['Point of Sale'])]
    df = df.loc[df['pos_location_name'].isin([sede])]

        # Convertir la columna 'day' a formato de fecha
    df['day'] = pd.to_datetime(df['day'], format='%Y-%m-%d')

    # Agrupar por día y product_id y sumar las cantidades
    df = df.groupby(['day', 'product_id', 'variant_sku']).sum().reset_index()
    
    def categorize_quantity(qty):
        if qty <= 2:
            return qty
        elif 3 <= qty < 7:
            return 1
        elif 7 <= qty <= 10:
            return 2
        elif qty >= 11:
            return 3

    df['net_quantity'] = df['net_quantity'].apply(categorize_quantity)

    # Agregar una columna 'month_name' con los nombres de los meses
    df['month_name'] = df['day'].dt.strftime('%B')

    # Pivotear el DataFrame para tener los nombres de los meses como columnas
    pivot_df = df.pivot_table(index=['product_id','variant_sku'], columns='month_name', values='net_quantity', aggfunc='sum', fill_value=0).reset_index()

    # Reorganizar el DataFrame con las columnas por meses
    ordered_columns = ["product_id", "variant_sku"] + months 
    pivot_df = pivot_df[ordered_columns]

    # Organizar el DataFrame pivotado según la cantidad total de ventas por SKU
    pivot_df['Total'] = pivot_df[months].sum(axis=1)
    pivot_df = pivot_df.sort_values(by='Total', ascending=False)  # Ordenar de mayor a menor según la suma total

    # Calcular la cantidad de meses con ventas         
    pivot_df["MonthsWithSales"] = pivot_df.apply(lambda row: sum(1 for month in months if row[month] > 0), axis=1) 

    df_filtered = pivot_df[~(pivot_df[months] == 0).all(axis=1)]

    inventario = inventario2

    # Convierte ambas columnas a tipo de dato str
    #pivot_df['variant_sku'] = pivot_df['variant_sku'].astype(str)
    #inventario['Variant SKU'] = inventario['Variant SKU'].astype(str)

    # Convierte ambas columnas a tipo de dato str
    pivot_df['product_id'] = pivot_df['product_id'].astype('Int64')
    inventario['ID'] = inventario['ID'].astype('Int64')

    columnas_deseadas_inventario = ['ID', 'Vendor', 'Type', 'Variant SKU', 'Created At', 'Inventory Available: Bukz Tesoro', 'Inventory Available: Bukz Mattelsa', 'Inventory Available: Bukz Las Lomas', 'Inventory Available: Cedi Lomas']
    inventario = inventario[columnas_deseadas_inventario].copy()  # Añade .copy() aquí
    inventario[['Inventory Available: Bukz Tesoro', 'Inventory Available: Bukz Mattelsa', 'Inventory Available: Bukz Las Lomas', 'Inventory Available: Cedi Lomas']] = inventario[['Inventory Available: Bukz Tesoro', 'Inventory Available: Bukz Mattelsa', 'Inventory Available: Bukz Las Lomas', 'Inventory Available: Cedi Lomas']].apply(lambda x: abs(x))

    # Realiza el cruce con las columnas convertidas
    df_resultado = pd.merge(pivot_df, inventario, left_on='product_id', right_on='ID', how='left')
    df_resultado = df_resultado.dropna(subset=['Variant SKU'])

    # Convierte la columna "Created At" al tipo de datos datetime si aún no lo está
    df_resultado['Created At'] = pd.to_datetime(df_resultado['Created At'])

    # Obtiene la fecha actual
    fecha_actual = datetime.now()

    # Calcula la diferencia en meses entre la fecha actual y la columna "Created At" en formato decimal
    df_resultado['meses de creación'] = ((fecha_actual - df_resultado['Created At']).dt.total_seconds() / (30 * 24 * 60 * 60)).round(1)

    # Aplicar todas las reglas en secuencia:
    def apply_all_rules(df, months, inv_sede):
        month_columns = [col for col in df.columns if col in months]
        df['sugerido'] = np.nan
        df['tipo_caso'] = ""

        df = set_suggested_for_old_books(df)
        df = set_suggested_for_months_with_sales(df)
        df = set_suggested_for_recent_creation(df)
        df = set_suggested_for_intermittent_sales(df, month_columns)
        df = set_suggested_for_no_inventory(df, inv_sede)
        df = set_suggested_for_recent_creation_with_low_sales(df)
        df = set_suggested_for_average_sales(df, months)

        # Handle NaNs in 'sugerido'
        mask_sugerido_nan = df['sugerido'].isna()
        df.loc[mask_sugerido_nan, 'sugerido'] = df[mask_sugerido_nan].apply(compute_six_month_average, axis=1).round(0)
        df.loc[mask_sugerido_nan, 'tipo_caso'] = "Promedio de ventas de 6 meses"

        df.loc[df['sugerido'] == 0, 'sugerido'] = 1
        df = df.drop("intermittent", axis=1)

        return df

    df_resultado = apply_all_rules(df_resultado, months, inv_sede)

    # Crear la columna 'sugerido_2' inicialmente con NaNs
    df_resultado['sugerido_2'] = np.nan
    df_resultado['meses de inventario'] = ""

    # Aplicamos la función y asignamos los resultados al DataFrame
    df_resultado[['sugerido_2', 'meses de inventario']] = df_resultado.apply(compute_values, axis=1)

    # Realizar la operación de resta
    df_resultado['Estado_inventario'+sede] = df_resultado[inv_sede] - df_resultado['sugerido_2']
    
    df_resultado.to_excel(f"resultado_{sede}.xlsx")
    
    resultados[sede] = df_resultado.copy()
    
    print("Completo " + sede)

Completo Bukz Tesoro
Completo Bukz Las Lomas
Completo Bukz Mattelsa


In [8]:
df_Mattelsa = resultados['Bukz Mattelsa']
df_Lomas = resultados['Bukz Las Lomas']
df_Tesoro = resultados['Bukz Tesoro']

print(len(df_Mattelsa))
print(len(df_Lomas))
print(len(df_Tesoro))
print(len(df_Mattelsa)+len(df_Lomas)+len(df_Tesoro))

582
3702
3216
7500


In [9]:
df_Tesoro = df_Tesoro.drop_duplicates(subset=['product_id'])
df_Mattelsa = df_Mattelsa.drop_duplicates(subset=['product_id'])
df_Lomas = df_Lomas.drop_duplicates(subset=['product_id'])


In [10]:
df_Tesoro = df_Tesoro[df_Tesoro['meses de creación'] > 1]
df_Mattelsa = df_Mattelsa[df_Mattelsa['meses de creación'] > 1]
df_Lomas = df_Lomas[df_Lomas['meses de creación'] > 1]


In [11]:
result = pd.concat([df_Mattelsa, df_Lomas, df_Tesoro])

In [16]:
result.to_excel("Df.xlsx")

In [15]:
def first_non_nan(series):
    return series.dropna().iloc[0] if not series.dropna().empty else np.nan

# Obtener valores únicos para las columnas de interés
unique_values = result.groupby('product_id').agg({
    'Estado_inventarioBukz Tesoro': first_non_nan,
    'Estado_inventarioBukz Mattelsa': first_non_nan,
    'Estado_inventarioBukz Las Lomas': first_non_nan,
}).reset_index()

# Eliminar duplicados y combinar con los valores únicos
result = result.drop_duplicates(subset='product_id').drop(columns=['Estado_inventarioBukz Tesoro','Estado_inventarioBukz Mattelsa', 'Estado_inventarioBukz Las Lomas'])
result = pd.merge(result, unique_values, on='product_id')


In [ ]:
print(result['product_id'].dtypes)
print(unique_values['product_id'].dtypes)


In [17]:
df = result.copy()
df = df.rename(columns={'Estado_inventarioBukz Las Lomas':'Lomas', 'Estado_inventarioBukz Tesoro':'Tesoro', 'Estado_inventarioBukz Mattelsa':'Mattelsa'})
bodegas = ['Lomas', 'Tesoro', 'Mattelsa']
df = df.drop_duplicates()
# Agregamos las columnas de transferencia y devolución
for i in bodegas:
    for j in bodegas:
        if i != j:
            df[f"De {i} a {j}"] = 0.0
    df[f"Devolver de {i}"] = 0.0

for index, row in df.iterrows():
    while True:
        deficit = {bodega: row[bodega] for bodega in bodegas if row[bodega] < 0}
        exceso = {bodega: row[bodega] for bodega in bodegas if row[bodega] > 0}

        # Si no hay bodegas en déficit o en exceso, rompemos el ciclo
        if not deficit or not exceso:
            break

        # Tomamos la bodega con mayor déficit y la de mayor exceso
        bodega_deficit = min(deficit, key=deficit.get)
        bodega_exceso = max(exceso, key=exceso.get)

        # Calculamos cuánto mover
        cantidad_mover = min(abs(row[bodega_deficit]), row[bodega_exceso])

        # Actualizamos la columna de transferencia y los valores de las bodegas
        df.at[index, f"De {bodega_exceso} a {bodega_deficit}"] += cantidad_mover
        row[bodega_deficit] += cantidad_mover
        row[bodega_exceso] -= cantidad_mover

# Actualización de la cantidad a devolver teniendo en cuenta lo enviado
for bodega in bodegas:
    libros_enviados = df[[col for col in df.columns if col.startswith(f"De {bodega}")]].sum(axis=1)
    df[f"Devolver de {bodega}"] = df[bodega] - libros_enviados
    df[f"Devolver de {bodega}"] = df[f"Devolver de {bodega}"].apply(lambda x: x if x > 0 else 0)
    
# 1. Suma todas las transferencias por bodega
transferencias_lomas = df['De Mattelsa a Lomas'] + df['De Tesoro a Lomas']
transferencias_tesoro = df['De Mattelsa a Tesoro'] + df['De Lomas a Tesoro']
transferencias_mattelsa = df['De Lomas a Mattelsa'] + df['De Tesoro a Mattelsa']

# 2. Resta estas sumas del inventario original para obtener el inventario final
df['Lomas final'] = df['Lomas'] + transferencias_lomas
df['Tesoro final'] = df['Tesoro'] + transferencias_tesoro
df['Mattelsa final'] = df['Mattelsa'] + transferencias_mattelsa

# 3. Determina cuántos libros hay que pedir basándote en los negativos del inventario final
df['Pedir Lomas'] = df['Lomas final'].apply(lambda x: abs(x) if x < 0 else 0)
df['Pedir Tesoro'] = df['Tesoro final'].apply(lambda x: abs(x) if x < 0 else 0)
df['Pedir Mattelsa'] = df['Mattelsa final'].apply(lambda x: abs(x) if x < 0 else 0)


In [19]:
df.to_excel("datos.xlsx")

In [21]:
# Definición de condiciones y asignaciones para cada sede
sedes = {
    'Mattelsa': 'De Mattelsa a CEDI',
    'Las Lomas': 'De Lomas a CEDI',
    'Tesoro': 'De Tesoro a CEDI'
}

for sede, new_col in sedes.items():
    condition = result[f'Estado_inventarioBukz {sede}'] > 0
    result[new_col] = np.where(condition, result[f'Estado_inventarioBukz {sede}'].abs(), np.nan)


In [22]:
result['devolver_a_CEDI'] = result[['De Mattelsa a CEDI', 'De Lomas a CEDI', 'De Tesoro a CEDI']].sum(axis=1)

In [23]:
result['total_CEDI'] = result[['devolver_a_CEDI','Inventory Available: Cedi Lomas']].sum(axis=1)

In [24]:
# Crear columnas para rastrear los libros enviados de CEDI a cada sede
for col in result.columns:
    if 'Estado_inventarioBukz' in col:
        result[f'De CEDI_a_{col.split()[-1]}'] = 0

# Crear una copia del total_CEDI para trabajar sin modificar la original
result['total_CEDI_copia'] = result['total_CEDI']

for idx, row in result.iterrows():
    # Obtener los déficits
    deficits = {col: row[col] for col in result.columns if 'Estado_inventarioBukz' in col and row[col] < 0}
    total_deficit = sum(abs(value) for value in deficits.values())

    # Cuántos libros realmente se pueden enviar desde CEDI para este row
    dispatchable_books = min(row['total_CEDI_copia'], total_deficit)
    
    # Si no hay libros para enviar, continuar con la siguiente fila
    if dispatchable_books == 0:
        continue

    for col, deficit in deficits.items():
        # Calcular la proporción del déficit de esta sede
        proportion = abs(deficit) / total_deficit

        # Calcular la cantidad de libros a enviar basados en la proporción
        amount_to_send = int(dispatchable_books * proportion)
        
        # Actualizar las columnas relevantes
        result.at[idx, col] += amount_to_send
        result.at[idx, f'De CEDI_a_{col.split()[-1]}'] += amount_to_send
        result.at[idx, 'total_CEDI_copia'] -= amount_to_send


In [25]:
for idx, _ in result.iterrows():
    # Identificar sedes con déficit y ordenarlas
    deficits = {col: result.at[idx, col] for col in result.columns if 'Estado_inventarioBukz' in col and result.at[idx, col] < 0}
    sorted_deficits = sorted(deficits, key=lambda k: deficits[k])
    
    # Distribuir libros desde CEDI a las sedes, de mayor a menor déficit
    for sede in sorted_deficits:
        if result.at[idx, 'total_CEDI_copia'] > 0:  # Si hay libros en CEDI
            result.at[idx, sede] += 1  # Sumar 1 libro al déficit de la sede
            result.at[idx, f'De CEDI_a_{sede.split()[-1]}'] += 1  # Registrar que se envió un libro desde CEDI a la sede
            result.at[idx, 'total_CEDI_copia'] -= 1  # Restar 1 libro de CEDI

In [26]:
# Agregar las nuevas columnas
for sede in ["Tesoro", "Mattelsa", "Las Lomas"]:
    result[f"pedir_{sede}"] = 0

# Actualizar las nuevas columnas según las condiciones
for idx, _ in result.iterrows():
    for sede in ["Tesoro", "Mattelsa", "Las Lomas"]:
        original_column = f"Estado_inventarioBukz {sede}"
        if result.at[idx, original_column] < 0:
            result.at[idx, f"pedir_{sede}"] = abs(result.at[idx, original_column])


In [27]:
selected_columns_mattelsa = ['product_id', 'variant_sku', 'Vendor','Inventory Available: Bukz Mattelsa','sugerido', 'meses de inventario', 
       'sugerido_2','Estado_inventarioBukz Mattelsa', 'meses de creación']
df_Mattelsa_new = df_Mattelsa[selected_columns_mattelsa].copy()

selected_columns_Tesoro = ['product_id', 'variant_sku', 'Vendor','Inventory Available: Bukz Tesoro','sugerido', 'meses de inventario', 
       'sugerido_2','Estado_inventarioBukz Tesoro', 'meses de creación']
df_Tesoro_new = df_Tesoro[selected_columns_Tesoro].copy()

selected_columns_lomas = ['product_id', 'variant_sku', 'Vendor','Inventory Available: Bukz Las Lomas','sugerido', 'meses de inventario', 
       'sugerido_2','Estado_inventarioBukz Las Lomas', 'meses de creación']
df_Lomas_new = df_Lomas[selected_columns_lomas].copy()

In [28]:
# Selecciona solo las columnas deseadas de df2
selected_columns = ['product_id', 'De Lomas a CEDI', 'De CEDI_a_Lomas', 'pedir_Las Lomas', 'total_CEDI_copia']
cruzar_lomas = result[selected_columns].copy()

# Realiza el merge
Lomas_final = pd.merge(df_Lomas_new, cruzar_lomas, on='product_id', how='left')

In [29]:
# Selecciona solo las columnas deseadas de df2
selected_columns_tesoro = ['product_id', 'De Tesoro a CEDI', 'De CEDI_a_Tesoro', 'pedir_Tesoro', 'total_CEDI_copia']
cruzar_tesoro = result[selected_columns_tesoro].copy()

# Realiza el merge
Tesoro_final = pd.merge(df_Tesoro_new, cruzar_tesoro, on='product_id', how='left')

In [30]:
# Selecciona solo las columnas deseadas de df2
selected_columns_mattelsa = ['product_id', 'De Mattelsa a CEDI', 'De CEDI_a_Mattelsa', 'pedir_Mattelsa', 'total_CEDI_copia']
cruzar_Mattelsa = result[selected_columns_mattelsa].copy()

# Realiza el merge
Mattelsa_final = pd.merge(df_Mattelsa_new, cruzar_Mattelsa, on='product_id', how='left')

In [31]:
def procesar_inventario(df_original, columna_inventario, df_final):
    vendors_excluidos = ['Bukz USA', 'Bukz España']
    
    # Filtrar el DataFrame por los criterios deseados
    filtered_df = df_original[df_original[columna_inventario].notna() & 
                              (df_original[columna_inventario] > 0) & 
                              (~df_original['Vendor'].isin(vendors_excluidos))]  # Esta línea excluye las filas con los Vendors especificados
    
    filtered_df = filtered_df[filtered_df['Type'].isin(['Libro', 'Libros', 'Libros impresos'])].copy()
    filtered_df[columna_inventario] = filtered_df[columna_inventario].apply(lambda x: 0 if x < 0 else x)
    filtered_df.rename(columns={'ID': 'product_id'}, inplace=True)
    
    no_ventas = filtered_df.merge(df_final, on='product_id', how='left', indicator=True)
    no_ventas = no_ventas[no_ventas['_merge'] == 'left_only']
    no_ventas = no_ventas.drop(columns=['_merge'])
    
    no_ventas['Created At'] = pd.to_datetime(no_ventas['Created At'])
    fecha_actual = datetime.now()
    no_ventas['meses de creación'] = ((fecha_actual - no_ventas['Created At']).dt.total_seconds() / (30 * 24 * 60 * 60)).round(1)
    
    return no_ventas

# Aplicar la función para cada sede
no_ventas_lomas = procesar_inventario(inventario_original, 'Inventory Available: Bukz Las Lomas', Lomas_final)
no_ventas_Tesoro = procesar_inventario(inventario_original, 'Inventory Available: Bukz Tesoro', Tesoro_final)
no_ventas_Mattelsa = procesar_inventario(inventario_original, 'Inventory Available: Bukz Mattelsa', Mattelsa_final)



In [33]:
def calcular_envio(row, col_inventario):
    meses = row["meses de creación"]
    inventario = row[col_inventario]
    
    if meses <= 1:
        return 0
    elif 1 < meses <= 3:
        if inventario > 3:
            return inventario - 3
        else:
            return 0
    elif 3 < meses <= 6:
        if inventario > 2:
            return inventario - 2
        else:
            return 0
    else: # si meses > 6
        if inventario > 1:
            return inventario - 1
        else:
            return 0

# Para Lomas
no_ventas_lomas["enviar_CEDI_no_ventas_Lomas"] = no_ventas_lomas.apply(calcular_envio, args=("Inventory Available: Bukz Las Lomas_x",), axis=1)

# Para Tesoro
no_ventas_Tesoro["enviar_CEDI_no_ventas_Tesoro"] = no_ventas_Tesoro.apply(calcular_envio, args=("Inventory Available: Bukz Tesoro_x",), axis=1)  # Asumiendo que esta es la columna correcta

# Para Mattelsa
no_ventas_Mattelsa["enviar_CEDI_no_ventas_Mattelsa"] = no_ventas_Mattelsa.apply(calcular_envio, args=("Inventory Available: Bukz Mattelsa_x",), axis=1)  # Asumiendo que esta es la columna correcta


In [34]:
# Concatenando los DataFrames
df_concatenado_Lomas = pd.concat([Lomas_final, no_ventas_lomas], ignore_index=True)
df_concatenado_Mattelsa = pd.concat([Mattelsa_final, no_ventas_Mattelsa], ignore_index=True)
df_concatenado_Tesoro = pd.concat([Tesoro_final, no_ventas_Tesoro], ignore_index=True)

In [35]:
# Extraer los 'variant_id' de cada DataFrame y convertirlos en conjuntos
ids_lomas = set(no_ventas_lomas['product_id'])
ids_Tesoro = set(no_ventas_Tesoro['product_id'])
ids_Mattelsa = set(no_ventas_Mattelsa['product_id'])

# Identificar los ids que están en al menos dos de los conjuntos
ids_repetidos = (ids_lomas & ids_Tesoro) | (ids_lomas & ids_Mattelsa) | (ids_Tesoro & ids_Mattelsa)


In [36]:
def actualizar_total(df_resultado, df_no_ventas, columna_enviar):
    """
    Esta función toma el DataFrame resultado y actualiza la columna 'total_CEDI_copia'
    sumando los valores de la columna enviar de df_no_ventas.
    """
    # Realizamos un merge en base a 'variant_id'
    merged_df = df_resultado.merge(df_no_ventas[['product_id', columna_enviar]], on='product_id', how='left')
    
    # Rellenamos los NaN con 0 (esto es para aquellos variant_id que no existen en df_no_ventas)
    merged_df[columna_enviar].fillna(0, inplace=True)
    
    # Sumamos los valores al total
    merged_df['total_CEDI_copia'] += merged_df[columna_enviar]
    
    # Eliminamos la columna temporal
    merged_df.drop(columns=columna_enviar, inplace=True)
    
    # Extraemos los registros que no cruzaron con df_principal_copia
    nuevos_registros = df_no_ventas[~df_no_ventas['product_id'].isin(merged_df['product_id'])].copy()
    nuevos_registros['total_CEDI_copia'] = nuevos_registros[columna_enviar]
    nuevos_registros = nuevos_registros[['product_id', 'total_CEDI_copia']]
    
    # Concatenamos al final de merged_df
    merged_df = pd.concat([merged_df, nuevos_registros], ignore_index=True)
    
    return merged_df

# Creando una copia de df_principal y seleccionando las columnas deseadas
columnas_deseadas = ['total_CEDI_copia', 'product_id']  # Agrega las columnas que desees conservar
df_principal_copia = result[columnas_deseadas].copy()

# Aplicamos la función para cada DataFrame
df_resultado = actualizar_total(df_principal_copia, no_ventas_lomas, 'enviar_CEDI_no_ventas_Lomas')
df_resultado = actualizar_total(df_resultado, no_ventas_Tesoro, 'enviar_CEDI_no_ventas_Tesoro')
df_resultado = actualizar_total(df_resultado, no_ventas_Mattelsa, 'enviar_CEDI_no_ventas_Mattelsa')



In [ ]:
def sugerencia_entre_sedes(df_origen, df_destino, inventario_df, columna_inventario, nombre_sede):
    # Cruce de registros de df_origen que no están en df_destino
    no_en_destino = df_origen[~df_origen['product_id'].isin(df_destino['product_id'])].copy()

    # Consultar inventario para esos registros
    no_en_destino = no_en_destino.merge(inventario_df[['ID', columna_inventario]], left_on='product_id', right_on='ID', how='left')
    no_en_destino['sugerido entre sedes'] = no_en_destino.apply(lambda row: 'enviar' if pd.isna(row[columna_inventario]) or row[columna_inventario] <= 0 else '', axis=1)
    no_en_destino = no_en_destino[no_en_destino['sugerido entre sedes'] == 'enviar']
    no_en_destino.drop(columns=['ID', columna_inventario], inplace=True)  # Eliminamos 'ID' y la columna de inventario
    no_en_destino['sede'] = nombre_sede
    
    return no_en_destino[['product_id', 'variant_sku', 'sugerido entre sedes', 'sede', 'sugerido']]



# Procesar cada combinación de DataFrames
sugerencia_Mattelsa_Tesoro = sugerencia_entre_sedes(Mattelsa_final, Tesoro_final, inventario_original, 'Inventory Available: Bukz Tesoro', 'Tesoro')
sugerencia_Mattelsa_Lomas = sugerencia_entre_sedes(Mattelsa_final, Lomas_final, inventario_original, 'Inventory Available: Bukz Las Lomas', 'Lomas')

sugerencia_Tesoro_Mattelsa = sugerencia_entre_sedes(Tesoro_final, Mattelsa_final, inventario_original, 'Inventory Available: Bukz Mattelsa', 'Mattelsa')
sugerencia_Tesoro_Lomas = sugerencia_entre_sedes(Tesoro_final, Lomas_final, inventario_original, 'Inventory Available: Bukz Las Lomas', 'Lomas')

sugerencia_Lomas_Tesoro = sugerencia_entre_sedes(Lomas_final, Tesoro_final, inventario_original, 'Inventory Available: Bukz Tesoro', 'Tesoro')
sugerencia_Lomas_Mattelsa = sugerencia_entre_sedes(Lomas_final, Mattelsa_final, inventario_original, 'Inventory Available: Bukz Mattelsa', 'Mattelsa')

# Consolidar los DataFrames para cada sede

df_Tesoro_2 = pd.concat([sugerencia_Mattelsa_Tesoro, sugerencia_Lomas_Tesoro], ignore_index=True)
df_Lomas_2 = pd.concat([sugerencia_Tesoro_Lomas, sugerencia_Mattelsa_Lomas], ignore_index=True)
df_Mattelsa_2 = pd.concat([sugerencia_Tesoro_Mattelsa, sugerencia_Lomas_Mattelsa], ignore_index=True)

# Para df_Tesoro_2
idx = df_Tesoro_2.groupby(['product_id'])['sugerido'].idxmin()
df_Tesoro_2 = df_Tesoro_2.loc[idx]

# para df_Lomas_2
idx_lomas = df_Lomas_2.groupby(['product_id'])['sugerido'].idxmin()
df_Lomas_2 = df_Lomas_2.loc[idx_lomas]

# para df_Mattelsa_2
idx_mattelsa = df_Mattelsa_2.groupby(['product_id'])['sugerido'].idxmin()
df_Mattelsa_2 = df_Mattelsa_2.loc[idx_mattelsa]

In [ ]:
df_concatenado = pd.concat([df_Tesoro_2, df_Mattelsa_2, df_Lomas_2], ignore_index=True)
df_concatenado = df_concatenado.drop_duplicates(subset=['product_id', 'sede'])

df_concatenado = df_concatenado.pivot(index='product_id', columns='sede', values='sugerido').reset_index()

# Rellenar valores NaN con 0
df_concatenado = df_concatenado.fillna(0).astype('Int64')


In [ ]:
# Convertir las columnas 'product_id' a Int64
df_concatenado['product_id'] = df_concatenado['product_id'].astype('Int64')
df_resultado['product_id'] = df_resultado['product_id'].astype('Int64')

# Hacer el merge
merged_df_sedes = pd.merge(df_concatenado, df_resultado, on='product_id', how='left')

In [ ]:

df_sedes = merged_df_sedes.copy()

df_sedes = merged_df_sedes.copy()

# Crear columnas para almacenar la cantidad de items enviados desde el CEDI a cada tienda
for tienda in ['Lomas', 'Mattelsa', 'Tesoro']:
    df_sedes[f'De_CEDI_a_{tienda}'] = 0
    df_sedes[f'Pedir_{tienda}'] = 0  # Columnas para almacenar cuánto más hay que pedir por sede

# Iterar sobre cada fila del DataFrame
for idx, row in df_sedes.iterrows():
    cedi_disponible = row['total_CEDI_copia']

    while cedi_disponible > 0 and any(row[['Lomas', 'Mattelsa', 'Tesoro']] > 0):
        for tienda in ['Lomas', 'Mattelsa', 'Tesoro']:
            if row[tienda] > 0 and cedi_disponible > 0:
                df_sedes.at[idx, tienda] -= 1
                cedi_disponible -= 1
                
                df_sedes.at[idx, f'De_CEDI_a_{tienda}'] += 1
                
                df_sedes.at[idx, 'total_CEDI_copia'] = cedi_disponible
                row = df_sedes.loc[idx]

    # Llenar las columnas de 'pedir' después de distribuir lo que se pudo desde el CEDI
    for tienda in ['Lomas', 'Mattelsa', 'Tesoro']:
        # Se calcula cuánto más hay que pedir por sede, teniendo en cuenta los ya enviados desde el CEDI
        df_sedes.at[idx, f'Pedir_{tienda}'] = df_sedes.at[idx, tienda] - df_sedes.at[idx, f'De_CEDI_a_{tienda}']


        
df_sedes = df_sedes[["product_id", "Lomas", "Mattelsa", "Tesoro", "total_CEDI_copia", "De_CEDI_a_Lomas", "De_CEDI_a_Mattelsa",  
                     "De_CEDI_a_Tesoro", "Pedir_Lomas", "Pedir_Mattelsa", "Pedir_Tesoro"]]

df_sedes = df_sedes[["product_id", "Lomas", "Mattelsa", "Tesoro", "total_CEDI_copia", "De_CEDI_a_Lomas", "De_CEDI_a_Mattelsa",  
                     "De_CEDI_a_Tesoro", "Pedir_Lomas", "Pedir_Mattelsa", "Pedir_Tesoro"]]


df1_unique = df1.drop_duplicates(subset=['product_id'])
merged_sedes = pd.merge(df_sedes, df1_unique[['product_id', 'product_title', 'variant_sku', 'product_vendor']], on='product_id', how='left')
merged_sedes = merged_sedes.rename(columns={'product_id': 'ID', 'product_title':'Titulo','variant_sku':'SKU', 'product_vendor':'Vendor'})


In [ ]:
df_distribucion_Lomas = merged_sedes[['ID','Titulo', 'SKU', 'Vendor', 'Lomas', 'De_CEDI_a_Lomas', 'Pedir_Lomas']].copy()
df_distribucion_Lomas = df_distribucion_Lomas[df_distribucion_Lomas['Lomas'] != 0].reset_index(drop=True)
df_distribucion_Lomas = df_distribucion_Lomas.rename(columns={'Lomas': 'Sugerido'})
df_distribucion_Lomas = df_distribucion_Lomas.sort_values(by='Sugerido', ascending=False)


df_distribucion_Mattelsa = merged_sedes[['ID','Titulo', 'SKU', 'Vendor', 'Mattelsa', 'De_CEDI_a_Mattelsa', 'Pedir_Mattelsa']].copy()
df_distribucion_Mattelsa = df_distribucion_Mattelsa[df_distribucion_Mattelsa['Mattelsa'] != 0].reset_index(drop=True)
df_distribucion_Mattelsa = df_distribucion_Mattelsa.rename(columns={'Mattelsa': 'Sugerido'})
df_distribucion_Mattelsa = df_distribucion_Mattelsa.sort_values(by='Sugerido', ascending=False)


df_distribucion_Tesoro = merged_sedes[['ID','Titulo', 'SKU', 'Vendor', 'Tesoro', 'De_CEDI_a_Tesoro', 'Pedir_Tesoro']].copy()
df_distribucion_Tesoro = df_distribucion_Tesoro[df_distribucion_Tesoro['Tesoro'] != 0].reset_index(drop=True)
df_distribucion_Tesoro = df_distribucion_Tesoro.rename(columns={'Tesoro': 'Sugerido'})
df_distribucion_Tesoro = df_distribucion_Tesoro.sort_values(by='Sugerido', ascending=False)


In [ ]:
def rename_columns(df, column_mapping):
    df.rename(columns=column_mapping, inplace=True)
    
def drop_columns(df, columns_to_drop):
    columns_to_drop_actual = [col for col in columns_to_drop if col in df.columns]
    return df.drop(columns_to_drop_actual, axis=1)

# Mapeo para Lomas_final
lomas_mapping = {'product_id': 'ID','variant_sku': 'SKU', 'Inventory Available: Bukz Las Lomas': 'Inventario_Actual','sugerido': 'Sugerido_Mensual',
    'meses de inventario': 'Meses_de_Inventario','sugerido_2': 'Sugerido_Final','Estado_inventarioBukz Las Lomas': 'Estado_Inventario','De Lomas a CEDI': 'De_Lomas_a_CEDI',
    'De CEDI_a_Lomas': 'De CEDI_a_Lomas','pedir_Las Lomas': 'Pedir_Lomas'}

# Mapeo para Tesoro_final
tesoro_mapping = {'product_id': 'ID','variant_sku': 'SKU','Inventory Available: Bukz Tesoro': 'Inventario_Actual','sugerido': 'Sugerido_Mensual',
    'meses de inventario': 'Meses_de_Inventario','sugerido_2': 'Sugerido_Final','Estado_inventarioBukz Tesoro': 'Estado_Inventario','De Tesoro a CEDI': 'De_Tesoro_a_CEDI',
    'De CEDI_a_Tesoro': 'De CEDI_a_Tesoro','pedir_Tesoro': 'Pedir_Tesoro'}

# Mapeo para Mattelsa_final
mattelsa_mapping = {'product_id': 'ID', 'variant_sku': 'SKU', 'Inventory Available: Bukz Mattelsa': 'Inventario_Actual', 'sugerido': 'Sugerido_Mensual',
    'meses de inventario': 'Meses_de_Inventario','sugerido_2': 'Sugerido_Final','Estado_inventarioBukz Mattelsa': 'Estado_Inventario','De Mattelsa a CEDI': 'De_Mattelsa_a_CEDI',
    'De CEDI_a_Mattelsa': 'De CEDI_a_Mattelsa','pedir_Mattelsa': 'Pedir_Mattelsa'}

# Renombrar columnas usando la función y mapeos
rename_columns(Lomas_final, lomas_mapping)
rename_columns(Tesoro_final, tesoro_mapping)
rename_columns(Mattelsa_final, mattelsa_mapping)

# Definir las columnas a eliminar
columns_to_drop = ['meses de creación', 'total_CEDI_copia']

# Aplicar la función
Lomas_final = drop_columns(Lomas_final, columns_to_drop)
Tesoro_final = drop_columns(Tesoro_final, columns_to_drop)
Mattelsa_final = drop_columns(Mattelsa_final, columns_to_drop)

In [ ]:
def rename_columns(df, column_mapping):
    df_copy = df.copy()
    df_copy.rename(columns=column_mapping, inplace=True)
    return df_copy


df_no_ventas_Lomas = no_ventas_lomas[['product_id', 'Variant SKU','Inventory Available: Bukz Las Lomas_x','Created At', 
                                      'enviar_CEDI_no_ventas_Lomas']]
lomas_mapping2 = {'product_id':'ID', 'Variant SKU': 'SKU', 'Inventory Available: Bukz Las Lomas_x': 'Inventario_Actual',
                 'Created At':'Fecha_Creación', 'enviar_CEDI_no_ventas_Lomas':'De_Lomas_a_CEDI'}
df_no_ventas_Lomas = rename_columns(df_no_ventas_Lomas, lomas_mapping2)
df_no_ventas_Lomas['Fecha_Creación'] = pd.to_datetime(df_no_ventas_Lomas['Fecha_Creación']).dt.date



df_no_ventas_Tesoro = no_ventas_Tesoro[['product_id', 'Variant SKU','Inventory Available: Bukz Tesoro_x','Created At', 
                                      'enviar_CEDI_no_ventas_Tesoro']]
tesoro_mapping2 = {'product_id':'ID', 'Variant SKU': 'SKU', 'Inventory Available: Bukz Tesoro_x': 'Inventario_Actual',
                 'Created At':'Fecha_Creación', 'enviar_CEDI_no_ventas_Tesoro':'De_Tesoro_a_CEDI'}
df_no_ventas_Tesoro = rename_columns(df_no_ventas_Tesoro, tesoro_mapping2)
df_no_ventas_Tesoro['Fecha_Creación'] = pd.to_datetime(df_no_ventas_Tesoro['Fecha_Creación']).dt.date


df_no_ventas_Mattelsa = no_ventas_Mattelsa[['product_id', 'Variant SKU','Inventory Available: Bukz Mattelsa_x','Created At', 
                                      'enviar_CEDI_no_ventas_Mattelsa']]
mattelsa_mapping2 = {'product_id':'ID', 'Variant SKU': 'SKU', 'Inventory Available: Bukz Mattelsa_x': 'Inventario_Actual',
                 'Created At':'Fecha_Creación', 'enviar_CEDI_no_ventas_Mattelsa':'De_Mattelsa_a_CEDI'}
df_no_ventas_Mattelsa = rename_columns(df_no_ventas_Mattelsa, mattelsa_mapping2)
df_no_ventas_Mattelsa['Fecha_Creación'] = pd.to_datetime(df_no_ventas_Mattelsa['Fecha_Creación']).dt.date


In [ ]:
df_sede_copy = df_sedes.copy()

df_sede_copy['total_enviado'] = df_sede_copy['De_CEDI_a_Lomas'] + df_sede_copy['De_CEDI_a_Mattelsa'] + df_sede_copy['De_CEDI_a_Tesoro']

df_sede_copy = df_sede_copy[['product_id', 'total_enviado']]

In [ ]:
df_sede_copy['product_id'] = df_sede_copy['product_id'].astype('Int64')
df_resultado['product_id'] = df_resultado['product_id'].astype('Int64')

merged_df_sedes_inv = pd.merge(df_resultado, df_sede_copy, on='product_id', how='left')

merged_df_sedes_inv['total_CEDI_copia_updated'] = merged_df_sedes_inv.apply(
    lambda row: row['total_CEDI_copia'] if pd.isna(row['total_enviado']) else row['total_CEDI_copia'] - row['total_enviado'],
    axis=1)

In [ ]:
inv_cedi = merged_df_sedes_inv[['product_id','total_CEDI_copia_updated']].copy()
inv_cedi['product_id'] = inv_cedi['product_id'].astype('Int64')

mask_type = inventario_original['Type'] == 'Libro'
mask_vendor = ~inventario_original['Vendor'].isin(['Bukz USA', 'Bukz España'])
inv_cei_tem = inventario_original[mask_type & mask_vendor][['ID', 'Inventory Available: Cedi Lomas']].copy()
inv_cei_tem['ID'] = inv_cei_tem['ID'].astype('Int64')

In [ ]:
inv_cei_tem_filtered = inv_cei_tem[inv_cei_tem['Inventory Available: Cedi Lomas'] > 0]

inv_cedi_general = pd.merge(inv_cedi, inv_cei_tem_filtered, left_on='product_id', right_on='ID', how='outer')

inv_cedi_general['ID'].fillna(inv_cedi_general['product_id'], inplace=True)
inv_cedi_general['total_CEDI_copia_updated'].fillna(inv_cedi_general['Inventory Available: Cedi Lomas'], inplace=True)

inv_cedi_general.drop(columns=['product_id','Inventory Available: Cedi Lomas'], inplace=True)
inv_cedi_general = inv_cedi_general[inv_cedi_general['total_CEDI_copia_updated'] > 0]


In [ ]:
df_no_ventas_Lomas['ID'] = df_no_ventas_Lomas['ID'].astype('Int64')
Lomas_final['ID'] = Lomas_final['ID'].astype('Int64')
df_distribucion_Lomas['ID'] = df_distribucion_Lomas['ID'].astype('Int64')
df_comcat_lomas = pd.concat([df_no_ventas_Lomas[['ID']], Lomas_final[['ID']], df_distribucion_Lomas[['ID']]], ignore_index=True).drop_duplicates()


df_no_ventas_Tesoro['ID'] = df_no_ventas_Tesoro['ID'].astype('Int64')
Tesoro_final['ID'] = Tesoro_final['ID'].astype('Int64')
df_distribucion_Tesoro['ID'] = df_distribucion_Tesoro['ID'].astype('Int64')
df_comcat_tesoro = pd.concat([df_no_ventas_Tesoro[['ID']], Tesoro_final[['ID']], df_distribucion_Tesoro[['ID']]], ignore_index=True).drop_duplicates()


df_no_ventas_Mattelsa['ID'] = df_no_ventas_Mattelsa['ID'].astype('Int64')
Mattelsa_final['ID'] = Mattelsa_final['ID'].astype('Int64')
df_distribucion_Mattelsa['ID'] = df_distribucion_Mattelsa['ID'].astype('Int64')
df_comcat_matt = pd.concat([df_no_ventas_Mattelsa[['ID']], Mattelsa_final[['ID']], df_distribucion_Mattelsa[['ID']]], ignore_index=True).drop_duplicates()


In [ ]:
df_concat_matt_no_match = pd.merge(inv_cedi_general, df_comcat_matt,  on='ID', how='left',  indicator=True)
df_concat_matt_no_match = df_concat_matt_no_match[df_concat_matt_no_match['_merge'] == 'left_only'].drop(columns='_merge')
df_concat_matt_no_match["sede"] = "Mattelsa"

df_concat_tesoro_no_match = pd.merge(inv_cedi_general, df_comcat_tesoro,  on='ID', how='left',  indicator=True)
df_concat_tesoro_no_match = df_concat_tesoro_no_match[df_concat_tesoro_no_match['_merge'] == 'left_only'].drop(columns='_merge')
df_concat_tesoro_no_match["sede"] = "Tesoro"

df_concat_lomas_no_match = pd.merge(inv_cedi_general, df_comcat_lomas,  on='ID', how='left',  indicator=True)
df_concat_lomas_no_match = df_concat_lomas_no_match[df_concat_lomas_no_match['_merge'] == 'left_only'].drop(columns='_merge')
df_concat_lomas_no_match["sede"] = "Lomas"

In [ ]:
frames = [df_concat_lomas_no_match, df_concat_tesoro_no_match, df_concat_matt_no_match]
df_concatenado_frames = pd.concat(frames, ignore_index=True)
df_concatenado_frames = df_concatenado_frames.drop_duplicates(subset=['ID', 'sede'])
df_concatenado_frames['presencia'] = 1
df_pivot_presencia = df_concatenado_frames.pivot_table(index='ID', columns='sede', values='presencia', fill_value=0).reset_index()
df_pivot_presencia_cedi = df_concatenado_frames.groupby('ID')['total_CEDI_copia_updated'].first().reset_index()
df_pivot_presencia_cedi = pd.merge(df_pivot_presencia_cedi, df_pivot_presencia, on='ID', how='outer')

mask = ~((df_pivot_presencia_cedi[['Lomas', 'Mattelsa', 'Tesoro']].sum(axis=1) == 3) & 
         (df_pivot_presencia_cedi['total_CEDI_copia_updated'].isin([1, 2]))) | (
        (df_pivot_presencia_cedi[['Lomas', 'Mattelsa', 'Tesoro']].sum(axis=1) == 2) &
        (df_pivot_presencia_cedi['total_CEDI_copia_updated'] == 1)
    )
df_pivot_presencia_cedi_2 = df_pivot_presencia_cedi[mask].copy()
# Función para determinar la cantidad de libros a enviar
def libros_a_enviar(row, col_name):
    if row['total_CEDI_copia_updated'] >= 6 and row[col_name] == 1:
        return 2
    elif row['total_CEDI_copia_updated'] < 6 and row[col_name] == 1:
        return 1
    else:
        return 0

# Crear nuevas columnas para cada sede, indicando cuántos libros enviar
sedes = ['Lomas', 'Mattelsa', 'Tesoro']
for sede in sedes:
    df_pivot_presencia_cedi_2[f'De_CEDI_a_{sede}'] = df_pivot_presencia_cedi_2.apply(lambda row: libros_a_enviar(row, sede), axis=1)

# Actualizando total_CEDI_copia_updated después de distribuir los libros
df_pivot_presencia_cedi_2['total_CEDI_copia_updated'] = df_pivot_presencia_cedi_2['total_CEDI_copia_updated'] - df_pivot_presencia_cedi_2[[f'De_CEDI_a_{sede}' for sede in sedes]].sum(axis=1)

# Asegurar que no hay valores negativos en 'total_CEDI_copia_updated'
df_pivot_presencia_cedi_2['total_CEDI_copia_updated'] = df_pivot_presencia_cedi_2['total_CEDI_copia_updated'].clip(lower=0)

In [ ]:
def filter_and_sort(df, sede_name, de_cedi_column):
    mask = df[sede_name] != 0
    filtered_df = df[mask].copy()
    filtered_df = filtered_df[['ID', de_cedi_column]]
    sorted_df = filtered_df.sort_values(by=de_cedi_column, ascending=False)
    return sorted_df

presencia_cedi_Tesoro = filter_and_sort(df_pivot_presencia_cedi_2, 'Tesoro', 'De_CEDI_a_Tesoro')
presencia_cedi_Lomas = filter_and_sort(df_pivot_presencia_cedi_2, 'Lomas', 'De_CEDI_a_Lomas')
presencia_cedi_Matelsa = filter_and_sort(df_pivot_presencia_cedi_2, 'Mattelsa', 'De_CEDI_a_Mattelsa')


In [ ]:
dfs = [Lomas_final, Tesoro_final, Mattelsa_final]

# Nombres originales de los DataFrames
df_names = ['Lomas_final', 'Tesoro_final', 'Mattelsa_final']

# Diccionario para almacenar los nuevos DataFrames
new_dfs = {}

# Crear y guardar un DataFrame "merged" para cada uno
for i, df in enumerate(dfs):
    merged_df = pd.merge(df, inventario_original[['ID', 'Title']], on='ID', how='left')
    merged_df.rename(columns={'Title': 'Titulo'}, inplace=True)
    
    # Creando el nuevo nombre y asignándolo al diccionario
    new_name = df_names[i] +'2'
    new_dfs[new_name] = merged_df

lomas_final_2 = new_dfs['Lomas_final2']
lomas_final_2 = lomas_final_2[["ID", "Titulo", "SKU", "Vendor", "Inventario_Actual", "Sugerido_Mensual", "Meses_de_Inventario", "Sugerido_Final", "Estado_Inventario", "De_Lomas_a_CEDI", "De CEDI_a_Lomas", "Pedir_Lomas"]]

tesoro_final_2 = new_dfs['Tesoro_final2']
tesoro_final_2 = tesoro_final_2[["ID", "Titulo", "SKU", "Vendor", "Inventario_Actual", "Sugerido_Mensual", "Meses_de_Inventario", "Sugerido_Final", "Estado_Inventario", "De_Tesoro_a_CEDI", "De CEDI_a_Tesoro", "Pedir_Tesoro"]]

mattelsa_final_2 = new_dfs['Mattelsa_final2']
mattelsa_final_2 = mattelsa_final_2[["ID", "Titulo", "SKU", "Vendor", "Inventario_Actual", "Sugerido_Mensual", "Meses_de_Inventario", "Sugerido_Final", "Estado_Inventario", "De_Mattelsa_a_CEDI", "De CEDI_a_Mattelsa", "Pedir_Mattelsa"]]


In [ ]:
dfs = [df_no_ventas_Mattelsa, df_no_ventas_Tesoro, df_no_ventas_Lomas]

# Nombres originales de los DataFrames
df_names = ['df_no_ventas_Mattelsa', 'df_no_ventas_Tesoro', 'df_no_ventas_Lomas']

# Diccionario para almacenar los nuevos DataFrames
new_dfs = {}

# Crear y guardar un DataFrame "merged" para cada uno
for i, df in enumerate(dfs):
    merged_df = pd.merge(df, inventario_original[['ID', 'Title', 'Vendor']], on='ID', how='left')
    merged_df.rename(columns={'Title': 'Titulo'}, inplace=True)
    
    # Creando el nuevo nombre y asignándolo al diccionario
    new_name = df_names[i] +'2'
    new_dfs[new_name] = merged_df

df_no_ventas_Mattelsa_2 = new_dfs['df_no_ventas_Mattelsa2']
df_no_ventas_Mattelsa_2 = df_no_ventas_Mattelsa_2[['ID', "Titulo", "SKU", "Vendor", 'Inventario_Actual', 'Fecha_Creación', 'De_Mattelsa_a_CEDI']]

df_no_ventas_Tesoro_2 = new_dfs['df_no_ventas_Tesoro2']
df_no_ventas_Tesoro_2 = df_no_ventas_Tesoro_2[['ID', "Titulo", "SKU", "Vendor", 'Inventario_Actual', 'Fecha_Creación', 'De_Tesoro_a_CEDI']]

df_no_ventas_Lomas_2 = new_dfs['df_no_ventas_Lomas2']
df_no_ventas_Lomas_2 = df_no_ventas_Lomas_2[['ID', "Titulo", "SKU", "Vendor", 'Inventario_Actual', 'Fecha_Creación', 'De_Lomas_a_CEDI']]

In [ ]:
dfs = [presencia_cedi_Tesoro, presencia_cedi_Lomas , presencia_cedi_Matelsa]

# Nombres originales de los DataFrames
df_names = ['presencia_cedi_Tesoro', 'presencia_cedi_Lomas' , 'presencia_cedi_Matelsa']

# Diccionario para almacenar los nuevos DataFrames
new_dfs = {}

# Crear y guardar un DataFrame "merged" para cada uno
for i, df in enumerate(dfs):
    merged_df = pd.merge(df, inventario_original[['ID', 'Title', 'Variant SKU','Vendor']], on='ID', how='left')
    merged_df.rename(columns={'Title': 'Titulo', 'Variant SKU':'SKU'}, inplace=True)
    
    # Creando el nuevo nombre y asignándolo al diccionario
    new_name = df_names[i] +'2'
    new_dfs[new_name] = merged_df

presencia_cedi_Tesoro_2 = new_dfs['presencia_cedi_Tesoro2']
presencia_cedi_Tesoro_2 = presencia_cedi_Tesoro_2[['ID', "Titulo", "SKU", "Vendor", 'De_CEDI_a_Tesoro' ]]

presencia_cedi_Lomas_2 = new_dfs['presencia_cedi_Lomas2']
presencia_cedi_Lomas_2 = presencia_cedi_Lomas_2[['ID', "Titulo", "SKU", "Vendor", 'De_CEDI_a_Lomas']]

presencia_cedi_Matelsa_2 = new_dfs['presencia_cedi_Matelsa2']
presencia_cedi_Matelsa_2 = presencia_cedi_Matelsa_2[['ID', "Titulo", "SKU", "Vendor",'De_CEDI_a_Mattelsa' ]]

In [ ]:
planeta = pd.read_excel("CATALOGO BUKZ PLANETA.xlsx")
penguin = pd.read_excel("CATALOGO ACTUALIZADO 26.09.2023 PRH.xlsx")

In [ ]:
# Para el primer grupo de DataFrames (Lomas):
dfs_Lomas = [lomas_final_2, df_no_ventas_Lomas_2, df_distribucion_Lomas, presencia_cedi_Lomas_2]

concatenado_Lomas_final = pd.concat([df[['ID', 'SKU']] for df in dfs_Lomas], ignore_index=True)

# Para el segundo grupo de DataFrames (Tesoro):
dfs_Tesoro = [tesoro_final_2, df_no_ventas_Tesoro_2, df_distribucion_Tesoro, presencia_cedi_Tesoro_2]

concatenado_Tesoro_final = pd.concat([df[['ID', 'SKU']] for df in dfs_Tesoro], ignore_index=True)

# Para el tercer grupo de DataFrames (Mattelsa):
dfs_Mattelsa = [mattelsa_final_2, df_no_ventas_Mattelsa_2, df_distribucion_Mattelsa, presencia_cedi_Matelsa_2]

concatenado_Mattelsa_final = pd.concat([df[['ID', 'SKU']] for df in dfs_Mattelsa], ignore_index=True)

In [ ]:
planeta['EAN'] = pd.to_numeric(planeta['EAN'], errors='coerce')
planeta['EAN'] = planeta['EAN'].astype('Int64')

penguin['ISBN'] = pd.to_numeric(penguin['ISBN'], errors='coerce')
penguin['ISBN'] = penguin['ISBN'].astype('Int64')

In [ ]:
penguin = penguin[(penguin["Cantidad ATP (Disponibilidad)"] > 0)]
planeta = planeta[(planeta["STOCK"] > 0)]

In [ ]:
concatenado_Lomas_final['SKU'] = pd.to_numeric(concatenado_Lomas_final['SKU'], errors='coerce')
concatenado_Lomas_final['SKU'] = concatenado_Lomas_final['SKU'].astype('Int64')

concatenado_Tesoro_final['SKU'] = pd.to_numeric(concatenado_Tesoro_final['SKU'], errors='coerce')
concatenado_Tesoro_final['SKU'] = concatenado_Tesoro_final['SKU'].astype('Int64')

concatenado_Mattelsa_final['SKU'] = pd.to_numeric(concatenado_Mattelsa_final['SKU'], errors='coerce')
concatenado_Mattelsa_final['SKU'] = concatenado_Mattelsa_final['SKU'].astype('Int64')


In [ ]:
concatenados = {
    'Lomas': concatenado_Lomas_final,
    'Tesoro': concatenado_Tesoro_final,
    'Mattelsa': concatenado_Mattelsa_final
}

# Diccionarios para guardar los DataFrames de EANs e ISBNs no encontrados
not_found_planeta = {}
not_found_penguin = {}

for name, concat_df in concatenados.items():
    # Cruzamos con planeta
    merged_with_planeta = pd.merge(left=planeta, right=concat_df, left_on='EAN', right_on='SKU', how='left', indicator=True)
    not_in_concatenado_planeta = merged_with_planeta[merged_with_planeta['_merge'] == 'left_only']
    
    # Cruzamos con penguin
    merged_with_penguin = pd.merge(left=penguin, right=concat_df, left_on='ISBN', right_on='SKU', how='left', indicator=True)
    not_in_concatenado_penguin = merged_with_penguin[merged_with_penguin['_merge'] == 'left_only']
    
    # Guardamos los DataFrames en los diccionarios usando una clave compuesta por la sede y la editorial
    not_found_planeta[f'{name}_planeta'] = not_in_concatenado_planeta
    not_found_penguin[f'{name}_penguin'] = not_in_concatenado_penguin



In [ ]:
not_found_in_Lomas_planeta = not_found_planeta['Lomas_planeta']
not_found_in_Lomas_penguin = not_found_penguin['Lomas_penguin']

not_found_in_Tesoro_planeta = not_found_planeta['Tesoro_planeta']
not_found_in_Tesoro_penguin = not_found_penguin['Tesoro_penguin']

not_found_in_Mattelsa_planeta = not_found_planeta['Mattelsa_planeta']
not_found_in_Mattelsa_penguin = not_found_penguin['Mattelsa_penguin']


In [ ]:
def crear_sugerencias(df_origen, vendor_name):
    df_sugerido = df_origen[['EAN', 'TITULO', 'AUTOR', 'SELLO', 'PVP', 'STOCK']].copy()
    df_sugerido['sugerido'] = df_sugerido['STOCK'].apply(lambda x: 2 if x > 10 else 1)
    df_sugerido["Vendor"] = vendor_name
    return df_sugerido[['EAN', 'TITULO', 'AUTOR', 'Vendor', 'SELLO', 'PVP', 'STOCK','sugerido']]

sugerido_planeta_Lomas = crear_sugerencias(not_found_in_Lomas_planeta, 'Editorial Planeta')
sugerido_planeta_Tesoro = crear_sugerencias(not_found_in_Tesoro_planeta, 'Editorial Planeta')
sugerido_planeta_Mattelsa = crear_sugerencias(not_found_in_Mattelsa_planeta, 'Editorial Planeta')

dataframes = [sugerido_planeta_Lomas, sugerido_planeta_Tesoro, sugerido_planeta_Mattelsa] 

for df in dataframes:
    df.rename(columns={'EAN':'SKU', 'TITULO':'Titulo', 'AUTOR':'Autor','SELLO':'Editorial', 'sugerido':'Sugerido'}, inplace=True)
    df.drop(columns=['STOCK'], inplace=True)

In [ ]:
def crear_sugerencias(df_origen, vendor_name):
    df_sugerido = df_origen[['ISBN', 'Titulo', 'Autor', 'Sello Editorial', 'Cantidad ATP (Disponibilidad)', 'PVP Actual']].copy()
    df_sugerido['sugerido'] = df_sugerido['Cantidad ATP (Disponibilidad)'].apply(lambda x: 2 if x > 10 else 1)
    df_sugerido["Vendor"] = vendor_name
    return df_sugerido[['ISBN', 'Titulo', 'Autor', 'Vendor', 'Sello Editorial', 'Cantidad ATP (Disponibilidad)', 'PVP Actual', 'sugerido']]

sugerido_penguin_Lomas = crear_sugerencias(not_found_in_Lomas_penguin, 'Penguin RandomHouse')
sugerido_penguin_Tesoro = crear_sugerencias(not_found_in_Tesoro_penguin, 'Penguin RandomHouse')
sugerido_penguin_Mattelsa = crear_sugerencias(not_found_in_Mattelsa_penguin, 'Penguin RandomHouse')

dataframes = [sugerido_penguin_Lomas, sugerido_penguin_Tesoro, sugerido_penguin_Mattelsa] 

for df in dataframes:
    df.rename(columns={'ISBN':'SKU','Sello Editorial':'Editorial', 'sugerido':'Sugerido', 'PVP Actual':'PVP'}, inplace=True)
    df.drop(columns=['Cantidad ATP (Disponibilidad)'], inplace=True)

In [ ]:
df_planeta_penguin_Lomas = pd.concat([sugerido_planeta_Lomas, sugerido_penguin_Lomas], ignore_index=True)

df_planeta_penguin_Tesoro = pd.concat([sugerido_planeta_Tesoro, sugerido_penguin_Tesoro], ignore_index=True)

df_planeta_penguin_Mattelsa = pd.concat([sugerido_planeta_Mattelsa, sugerido_penguin_Mattelsa], ignore_index=True)

In [ ]:
with pd.ExcelWriter('Sugerido_Lomas_Nov2023.xlsx', engine='openpyxl') as writer:
    lomas_final_2.to_excel(writer, sheet_name='Con_Ventas', index=False)
    df_no_ventas_Lomas_2.to_excel(writer, sheet_name='No_ventas', index=False)
    df_distribucion_Lomas.to_excel(writer, sheet_name='Venta_entre_sedes', index=False)
    presencia_cedi_Lomas_2.to_excel(writer, sheet_name='Cedi_Inv', index=False)
    df_planeta_penguin_Lomas.to_excel(writer, sheet_name='Planeta_Penguin', index=False)

In [ ]:
with pd.ExcelWriter('Sugerido_Tesoro_Nov2023.xlsx', engine='openpyxl') as writer:
    tesoro_final_2.to_excel(writer, sheet_name='Con_Ventas', index=False)
    df_no_ventas_Tesoro_2.to_excel(writer, sheet_name='No_ventas', index=False)
    df_distribucion_Tesoro.to_excel(writer, sheet_name='Venta_entre_sedes', index=False)
    presencia_cedi_Tesoro_2.to_excel(writer, sheet_name='Cedi_Inv', index=False)
    df_planeta_penguin_Tesoro.to_excel(writer, sheet_name='Planeta_Penguin', index=False)

In [ ]:
with pd.ExcelWriter('Sugerido_Mattelsa_Nov2023.xlsx', engine='openpyxl') as writer:
    mattelsa_final_2.to_excel(writer, sheet_name='Con_Ventas', index=False)
    df_no_ventas_Mattelsa_2.to_excel(writer, sheet_name='No_ventas', index=False)
    df_distribucion_Mattelsa.to_excel(writer, sheet_name='Venta_entre_sedes', index=False)
    presencia_cedi_Matelsa_2.to_excel(writer, sheet_name='Cedi_Inv', index=False)
    df_planeta_penguin_Mattelsa.to_excel(writer, sheet_name='Planeta_Penguin', index=False)